In [42]:
import torch

#FUNCTION TO CREATE PARALLEL DB
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

def get_parallel_dbs(db):
    parallel_dbs = list()
    
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs
def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [43]:
db, pdbs = create_db_and_parallels(20)

In [44]:
db #DEMO

tensor([0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
       dtype=torch.uint8)

In [45]:
pdbs #DEMO

[tensor([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
        dtype=torch.uint8),
 tensor([0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,

In [46]:
def query(db):
    return db.sum()

In [47]:
query(db) #DEMO

tensor(8)

In [48]:
query(pdbs[0]) #DEMO. change index value to show diff values of sum for each parallel db in pdbs

tensor(8)

In [49]:
#TO FIND MAX VALUE BY WHICH OUTPUT OF PARALLEL DBS CHANGE WRT FULL DB

full_db_result = query(db)
sensitivity = 0

for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance
print(sensitivity)

tensor(1)


In [50]:
#PACKING ALL THE CODE INTO A FUNCTION

def sensitivity(query, n_entries):
    db, pdbs = create_db_and_parallels(n_entries)
    full_db_result = query(db)
    sensitivity = 0

    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)
    
        if(db_distance > sensitivity):
            sensitivity = db_distance
    return sensitivity

def query(db):
    return db.float().mean() #Average value in the database

In [51]:
sensitivity(query, 1000)

tensor(0.0005)

In [52]:
#NEW PROGRAM...USING THRESHOLD VALUES FOR QUERY FUNCTION

In [86]:
def query(db, threshold=5):
    return (db.sum() > threshold).float()    

In [87]:
for i in range(10):
    sens_f = sensitivity(query,10)
    print(sens_f)

0
0
0
0
tensor(1.)
0
tensor(1.)
0
0
0


In [88]:
#PROGRAM TO PERFORM DIFFERENCING ATTACK ON ROW 10

In [89]:
db, _  = create_db_and_parallels(100)
print(db[8])


tensor(1, dtype=torch.uint8)


In [90]:
pdb = get_parallel_db(db, 8)

In [91]:
pdb

tensor([1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
        1, 0, 1], dtype=torch.uint8)

In [92]:
#Differencing attack using sum() #Answer shud be same as db[10]
sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [93]:
#using mean()
(sum(db).float()/len(db)) - (sum(pdb).float()/len(pdb))

tensor(0.0052)

In [94]:
sum(db)

tensor(49, dtype=torch.uint8)

In [95]:
#sum(pdb) #TO CHECK THAT ELEMENT IS ACTUALLY REMOVED

In [96]:
#using threshold = sum-1
(sum(db).float()>48) - (sum(pdb).float()>48)

tensor(1, dtype=torch.uint8)